In [ ]:
import numpy as np
import collections
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import datetime  
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)
data_path = '../data-unversioned/p1ch7/'

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cpu.


In [ ]:
cifar10 = datasets.CIFAR10(data_path, train=True, download= True)           # Training dataset....
cifar10_val = datasets.CIFAR10(data_path, train=False, download= True)      # Dataset for validation....

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'] # class definations...

In [ ]:
# Splitting dataset into training and validation set...

# Trainig data...
transformed_cifar10_Train = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Validation data...
transformed_cifar10_Val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
# Class "Net32" for model defination...
class Net32(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, 4 * 4 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

# Traing loop function...
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_loader:  
            imgs = imgs.to(device=device)  
            labels = labels.to(device=device)
            outputs = model(imgs)  
            loss = loss_fn(outputs, labels)  
            optimizer.zero_grad()         
            loss.backward()         
            optimizer.step()  
            loss_train += loss.item()  

        if epoch == 1 or epoch % 30 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader))) 

# Function calculating the validation of the model...
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad(): 
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) 
                total += labels.shape[0]  
                correct += int((predicted == labels).sum())  

        print("Accuracy {}: {:.2f}".format(name , correct / total))

In [ ]:
# Training the model "Net32" with dataset divided into minibatches...
train_mbload = torch.utils.data.DataLoader(transformed_cifar10_Train, batch_size=64, shuffle=True)  

model = Net32().to(device = device) 
optimizer = optim.SGD(model.parameters(), lr=1e-2)  
loss_fn = nn.CrossEntropyLoss()  

training_loop( 
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_mbload,
)

2022-03-30 14:22:40.419924 Epoch 1, Training loss 2.134170754333896
2022-03-30 14:47:26.026076 Epoch 30, Training loss 0.9316333950023212
2022-03-30 15:12:54.661497 Epoch 60, Training loss 0.7968545273289351
2022-03-30 15:38:33.432606 Epoch 90, Training loss 0.7305309954659104
2022-03-30 16:04:15.446954 Epoch 120, Training loss 0.6876822870100856
2022-03-30 16:29:47.098999 Epoch 150, Training loss 0.6558524637347292
2022-03-30 16:55:28.416072 Epoch 180, Training loss 0.63074853719043
2022-03-30 17:21:23.946591 Epoch 210, Training loss 0.6114760314107246
2022-03-30 17:47:44.875344 Epoch 240, Training loss 0.5961492321908931
2022-03-30 18:14:08.230775 Epoch 270, Training loss 0.582650717757547
2022-03-30 18:40:22.125260 Epoch 300, Training loss 0.5723558143352914


In [ ]:
# Checking the accuracy of model...
train_accuload = torch.utils.data.DataLoader(transformed_cifar10_Train, batch_size = 64, shuffle=False)
Valid_accuload = torch.utils.data.DataLoader(transformed_cifar10_Val, batch_size = 64, shuffle=False)

validate(model, train_accuload , Valid_accuload)

Accuracy train: 0.79
Accuracy val: 0.67
